This notebook generates randomized dictionary objects representing artificial observation blocks as desribed in [these pages](https://keckobservatory.atlassian.net/wiki/spaces/DSI/pages/808321035/Change+controlled+documents). 

Data is stored on a local MongoDB collection.

Finally a few function calls are made to query observation blocks

In [2]:
import numpy as np
from controller_helper import *
import pymongo
import random
import string
from itertools import product
import pprint
import pdb
from functools import wraps
import urllib
from getpass import getpass
seed = 1984739
random.seed(seed)

INST_MAPPING = { 
                 'DEIMOS': {'DE', 'DF'},
                 'ESI': {'EI'},
                 'HIRES': {'HI'},
                 'KCWI': {'KB', 'KF'}, 
                 'LRIS': {'LB', 'LR'},
                 'MOSFIRE': {'MF'},
                 'OSIRIS': {'OI', 'OS'},
                 'NIRES': {'NR', 'NI', 'NS'},
                 'NIRC2': {'N2', 'NC'},
                }

observers = ['Butawhiteboy Cantbekhan',
             'Snorkeldink Crackerdong',
             'Blubberwhale Crimpysnitch', 
             'Oscarbait Rivendell',
             'Buckingham Countryside',
             'Beezlebub Vegemite',
             'Wimbledon Crackerjack',
            ]

pis = [
    'Darmond steelbreaker',
    'Thorgarn strongmane',
    'Melkyl kindpast',
    'Brannyl firststone',
    'Thergrun marblebottom',
    'Tornur fobrekahk',
    'Emnir throroguhr',
    'Balmun dekork',
    'Guldram brufdinack',
    'Gulkum rurdurarr',
]

semesters = [str(x)+y for x, y in product(range(2000,2004), ['A', 'B'])]
letters = string.ascii_lowercase

# random generators 
randString = lambda x=4: ''.join(random.choice(letters) for i in range(x))
randFloat = lambda mag=10: mag * random.uniform(0,1)
randBool = lambda: bool(random.choice([0,1, None]))
randArrStr = lambda x=1, y=1: [randString(x) for _ in range(random.randint(1, y)) ]
optionalRandString = lambda x=4: random.choice([None, randString(x)])
optionalRandArrString = lambda x, y=1: random.choice([None, randArrStr(x, y)])
sampleInst = lambda: random.choice(list(INST_MAPPING.keys()))
randPI = lambda: random.choice(pis)
randObserver = lambda: random.choice(observers)
randSemester = lambda: random.choice(semesters)
randPIList = lambda x=1: lambda x=1: list(np.random.choice(pis, size=random.randint(1, x), replace=False))
randObserverList = lambda x=1: list(np.random.choice(observers, size=random.randint(1, x), replace=False))

Functions below generate observation blocks dictionary objects, and their sub components.

In [3]:
def remove_none_values_in_dict(method):
    '''None values in dict returned by method are removed
    '''
    @wraps(method)
    def remove_none(*args, **kw):
        result = method(*args, **kw)
        return {key: val for key, val in result.items() if val is not None}
    return remove_none

def generate_semester(sem, nLen, maxLen=6):
    return {'_id': sem,
            'semester': sem,
            'obs_id': randObserverList(maxLen)}

def generate_semesters(nSem, nLen=5, maxLen=6):
    return [ generate_semester(sem, nLen, maxLen) for sem in semesters[0:nSem] ]

def generate_mag(nLen=2):
    return {'band': randString(nLen), 'mag': randFloat(nLen)}

@remove_none_values_in_dict
def generate_observation(nLen, maxArr):
    schema = {
        'instrument': sampleInst(),
        'exposure_sequences': randArrStr(nLen, maxArr),
        'associations': randArrStr(nLen, maxArr),
    }
    return schema

@remove_none_values_in_dict
def generate_signature(maxArr):
    schema = {
        'pi': randPI(),
        'semester': randSemester(),
        'program': random.randint(0, 10),
        'observers': randObserverList(maxArr),
        'container': random.randint(0,3) 
    }
    return schema

@remove_none_values_in_dict
def generate_acquisition(nLen, maxArr):
    schema = {
        'instrument_setup': randString(),
        'acquisition_method': randString(),
        'guider_selection': optionalRandString(),
        'ao_modes': optionalRandArrString(nLen, maxArr),
        'offset_stars': optionalRandArrString(nLen, maxArr),
        'slitmasks': optionalRandArrString(nLen, maxArr),
        'position_angles': optionalRandArrString(nLen, maxArr),
    }
    return schema

@remove_none_values_in_dict
def generate_target():
    schema = { 
        'name': randString(), 
        'ra': randString(), 
        'dec': randString(), 
        'equinox': randFloat(), 
        'frame': randString(), 
        'ra_offset': randFloat(), 
        'dec_offset': randFloat(), 
        'pm_ra': randFloat(), 
        'pm_dec': randFloat(), 
        'epoch': randFloat(), 
        'obstime': randFloat(), 
        'mag': generate_mag(), 
        'wrap': optionalRandString(), 
        'd_ra': randFloat(), 
        'd_dec': randFloat(), 
        'comment': optionalRandString(), 
             }
    return schema

@remove_none_values_in_dict
def generate_observation_block(nLen, maxArr, _id=None):
    schema = {
        'signature': generate_signature(maxArr),
        'target': random.choice( [ None, generate_target() ]) ,
        'acquisition': random.choice( [ None, generate_acquisition(nLen, maxArr) ] ) ,
        'observations': [ generate_observation(nLen, maxArr) for _ in range(random.randint(1, maxArr)) ], 
        'associations': randArrStr(nLen, maxArr),
        'priority': randFloat(100)
    }
    if _id:
        schema['_id'] = _id
    return schema

In [4]:
# run this a few times to check if data is generating.
assert generate_observation_block(3,3), 'ob did not generate successfully'

Database name is 'papahana' collection name is 'ob_block'. Create collection assumes to be assessed through port 27017.

In [7]:
dbName = 'papahana'
collName = 'ob_block'
remote=True # run on remote server (n)
coll = create_collection(dbName, collName, port=27017, remote=False)

In [8]:
coll.drop()
coll.create_index([('signature.pi', pymongo.DESCENDING)])
coll.create_index([('signature.semester', pymongo.DESCENDING)])
coll.create_index([('signature.program', pymongo.DESCENDING)])

'signature.program_-1'

In [9]:
nLen = 5
maxArr = 5
nOb = 100
for idx in range(nOb):
    doc = generate_observation_block(nLen, maxArr, str(idx))
    asdf = coll.insert_one(doc)

In [13]:
asdf.inserted_id

'99'

The following queries can be called by the API, returning dictionaries as JSON objects. The first objects of each query are printed out. You are encouraged to alter the search queries yourself. 

In [7]:
sig = { "pi" : "Thergrun marblebottom", "semester" : "2001B", "program" : 10, "observers": ['Blubberwhale Crimpysnitch'] }

container = 2

queries = (
    coll.find(),
    find_by_pi(sig["pi"], coll),
    find_by_observer(sig['observers'][0], coll),
    find_by_observer_container(sig['observers'][0], container, coll)
)

for cursor in queries:
    obExample = cursor.next()
    pprint.pprint(obExample, depth=2)
    

{'_id': '0',
 'associations': ['mrtkq', 'miqme', 'bsful', 'jbuyl'],
 'observations': [{...}, {...}, {...}, {...}],
 'priority': 14.898833693629953,
 'signature': {'container': 3,
               'observers': [...],
               'pi': 'Darmond steelbreaker',
               'program': 5,
               'semester': '2000A'}}
{'_id': '8',
 'acquisition': {'acquisition_method': 'kmux',
                 'ao_modes': [...],
                 'guider_selection': 'nrvs',
                 'instrument_setup': 'elbr',
                 'offset_stars': [...],
                 'position_angles': [...],
                 'slitmasks': [...]},
 'associations': ['xpihn', 'earlh', 'kllti', 'hknjx'],
 'observations': [{...}, {...}],
 'priority': 21.242674436362353,
 'signature': {'container': 0,
               'observers': [...],
               'pi': 'Thergrun marblebottom',
               'program': 5,
               'semester': '2002A'}}
{'_id': '3',
 'associations': ['kpkjn'],
 'observations': [{...}, {..

In [9]:
aggregates = (
get_distinct_semesters(sig['observers'][0], coll),  # /semIds/{semester} (get) 
get_semesters_by_pi(sig['pi'], coll),  # /semIds (get) # Retrieves all the programs (semesters) associated with a PI
get_ob_by_semester(sig['semester'], coll), #  /semIds/{semester} (get) retrieve the programs for the given semester
get_ob_by_semester_observer(sig['semester'], sig['observers'][0], coll)  # /semIds/{semId} (get)  Retrieves the specified program
)

for cursor in aggregates:
    obExample = cursor.next()
    pprint.pprint(obExample, depth=2)

{'_id': {'semester': '2002A'}}
{'_id': '8', 'signature': {'semester': '2002A'}}
{'_id': '6',
 'acquisition': {'acquisition_method': 'vqaz', 'instrument_setup': 'otcf'},
 'associations': ['ufguo', 'hrxvt', 'nmkdn', 'aenjw', 'oxtcv'],
 'observations': [{...}, {...}, {...}, {...}, {...}],
 'priority': 95.62990777131259,
 'signature': {'container': 3,
               'observers': [...],
               'pi': 'Melkyl kindpast',
               'program': 2,
               'semester': '2001B'}}
{'_id': '14',
 'acquisition': {'acquisition_method': 'zjkn',
                 'ao_modes': [...],
                 'guider_selection': 'zmjt',
                 'instrument_setup': 'nsnd',
                 'offset_stars': [...],
                 'position_angles': [...],
                 'slitmasks': [...]},
 'associations': ['oyblk'],
 'observations': [{...}],
 'priority': 70.01206700080458,
 'signature': {'container': 3,
               'observers': [...],
               'pi': 'Brannyl firststone',
      

In [10]:
ob = list(get_ob_by_id('17', coll))[0]
ob['_id'] = str(101)
delete_observation_block(str(101), coll)
insert_observation_block(ob, coll)
newValues = {"$set": { "priority": 100 } }
update_observation_block(str(101), newValues, coll)
ob = list(get_ob_by_id('101', coll))
print(ob)

[{'_id': '101', 'signature': {'pi': 'Darmond steelbreaker', 'semester': '2003A', 'program': 6, 'observers': ['Oscarbait Rivendell'], 'container': 2}, 'target': {'name': 'ycsz', 'ra': 'qzff', 'dec': 'jlrv', 'equinox': 2.0613887207324746, 'frame': 'lwxo', 'ra_offset': 4.559331678104458, 'dec_offset': 0.9132324667888048, 'pm_ra': 1.2738993028947598, 'pm_dec': 8.060415779485188, 'epoch': 2.0540388831409917, 'obstime': 2.5541241243502077, 'mag': {'band': 'ch', 'mag': 1.405319377374786}, 'd_ra': 1.8974480563535945, 'd_dec': 4.035972332394922}, 'observations': [{'instrument': 'ESI', 'exposure_sequences': ['tdgip', 'vtytw', 'rcela', 'bkajk'], 'associations': ['cvvdj', 'xlytz']}, {'instrument': 'KCWI', 'exposure_sequences': ['owcjo', 'vbmrq'], 'associations': ['mdkha', 'witmj']}, {'instrument': 'NIRC2', 'exposure_sequences': ['ztmlk', 'hqhzi', 'bnpkg', 'imzof'], 'associations': ['laxuh', 'yfsag', 'iiwuv', 'kefnf']}, {'instrument': 'NIRC2', 'exposure_sequences': ['vbtfy'], 'associations': ['eqkq